In [1]:
#!pip install pillow


[notice] A new release of pip is available: 22.2.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import math
from tkinter import *
from tkinter.filedialog import *
from tkinter.simpledialog import* #이동,축소
from PIL import Image
import copy
import numpy as np

In [3]:
## 함수 선언부
### 공통 함수부 ###
def loadImage():
    
    global window, canvas, paper, filename
    global inImage, outImage, inH, inW, outH, outW
    filename = askopenfilename(parent=window,filetypes=(("칼라","*.png;*.jpg;*.bmp;*.gif;"),("모든 파일", "*.*")))
    #파일 크기 알아내기
    pillow = Image.open(filename)
    inH = pillow.height
    inW = pillow.width
    #메모리 확보(영상 크기)
    inImage = [[[0 for _ in range(inW)] for _ in range(inH)]for _ in range(3)]
    #파일 --> 메모리 로딩
    pillowRGB = pillow.convert("RGB") #RGB모델로 변경
    for i in range(inH):
        for k in range(inW):
            r, g, b = pillowRGB.getpixel((k,i))
            inImage[0][i][k] = r
            inImage[1][i][k] = g
            inImage[2][i][k] = b
            
    equalImage()

def displayImage() : 
    global window, canvas, paper, filename
    global inImage, outImage, inH, inW, outH, outW

    if canvas != None :
        canvas.destroy()
    window.geometry(str(outW)+ "x" + str(outH))  #흑백이랑 달리 w, h 변경
    canvas = Canvas(window, height=outH, width=outW)
    paper = PhotoImage(height=outH, width=outW)
    canvas.create_image((outW/2, outH/2), image=paper, state="normal") #흑백이랑 달리 w, h 변경
    rgbString = ""            #큰 메모리
    for i in range(outH) :
        tmpString = ""        #한 줄
        for k in range(outW) :
            r = outImage[0][i][k]
            g = outImage[1][i][k]
            b = outImage[2][i][k]
            tmpString += '#%02x%02x%02x ' % (r, g, b) #"" 안에 한 칸 띄워야 함
        rgbString += '{' + tmpString + '} '           #여기도 마지막 "" 안에 한 칸 띄워야함
    paper.put(rgbString)
    
    canvas.pack()
    
#이미지 저장
def saveImage():
    global window, canvas, paper, filename
    global inImage, outImage, inH, inW, outH, outW
    saveFp = asksaveasfile(parent = window, mode="wb",
                          defaultextension="*.raw",
                           filetypes=(("RAW파일","*.raw"),("모든 파일","*.*")))
    import struct
    for i in range(outH) :
        for k in range(outW):
            saveFp.write( struct.pack("B", outImage[i][k]))
    saveFp.close()
    messagebox.showinfo("성공", saveFp.name + "으로 저장")


### 영상처리 함수부 ###    
#동일(원래)사진
def equalImage():
    global window, canvas, paper, filename
    global inImage, outImage, inH, inW, outH, outW
    #중요! 출력이미지의 크기를 결정 -> 알고리즘에 따라서
    outH = inH
    outW = inW
    #메모리 할당
    outImage = [[[0 for _ in range(outW)] for _ in range(outH)] for _ in range(3)]
    ## ** 찐 영상처리 알고리즘 ** ##
    for rgb in range(3):
        for i in range(inH):
            for k in range(inW):
                outImage[rgb][i][k] = inImage[rgb][i][k]
    ###########################
    displayImage()
    
#gray 이미지로 변환
def grayImage():
    global window, canvas, paper, filename
    global inImage, outImage, inH, inW, outH, outW
    #중요! 출력이미지의 크기를 결정 -> 알고리즘에 따라서
    outH = inH
    outW = inW
    #메모리 할당
    outImage = [[[0 for _ in range(outW)] for _ in range(outH)] for _ in range(3)]
    ## ** 찐 영상처리 알고리즘 ** ##
    for i in range(inH):
        for k in range(inW):
            hap = inImage[0][i][k] + inImage[1][i][k] + inImage[2][i][k]
            outImage[0][i][k] = outImage[1][i][k] = outImage[2][i][k] = hap//3
    ###########################
    displayImage()
    
#반전
def reverseImage():
    global window, canvas, paper, filename
    global inImage, outImage, inH, inW, outH, outW
    #중요! 출력이미지의 크기를 결정 -> 알고리즘에 따라서
    outH = inH
    outW = inW
    #메모리 할당
    outImage = [[[0 for _ in range(outW)] for _ in range(outH)] for _ in range(3)]

    ## ** 찐 영상처리 알고리즘 ** ##
    for rgb in range(3):
        for i in range(inH):
            for k in range(inW):
                outImage[rgb][i][k] = 255 - inImage[rgb][i][k]
    
    ###########################
    displayImage()

#밝게or어둡게
def addImage():
    global window, canvas, paper, filename
    global inImage, outImage, inH, inW, outH, outW
    #중요! 출력이미지의 크기를 결정 -> 알고리즘에 따라서
    outH = inH
    outW = inW
    #메모리 할당
    outImage = [[[0 for _ in range(outW)] for _ in range(outH)] for _ in range(3)]

    ## ** 찐 영상처리 알고리즘 ** ##
    value = askinteger("밝게할 값", "-255부터 255까지 입력", minvalue = -255, maxvalue = 255)
    for rgb in range(3):   
        for i in range(inH):
            for k in range(inW):
                if (inImage[rgb][i][k] + value > 255) :
                    outImage[rgb][i][k] = 255
                elif (inImage[rgb][i][k] + value < 0) :
                    outImage[rgb][i][k] = 0
                else :
                    outImage[rgb][i][k] = inImage[rgb][i][k] + value

    
    ###########################
    displayImage()

#흑백1 : 반 값 이용
def BWImage1():
    global window, canvas, paper, filename
    global inImage, outImage, inH, inW, outH, outW
    #중요! 출력이미지의 크기를 결정 -> 알고리즘에 따라서
    outH = inH
    outW = inW
    #메모리 할당
    outImage = [[[0 for _ in range(outW)] for _ in range(outH)] for _ in range(3)]

    ## ** 찐 영상처리 알고리즘 ** ##
    for rgb in range(3):     
        for i in range(inH):
            for k in range(inW):
                if inImage[rgb][i][k] > 127 :
                    outImage[rgb][i][k] = 255
                else:    
                    outImage[rgb][i][k] = 0

    ###########################
    displayImage()

#흑백2 : 평균값
def BWImage2():
    global window, canvas, paper, filename
    global inImage, outImage, inH, inW, outH, outW
    #중요! 출력이미지의 크기를 결정 -> 알고리즘에 따라서
    outH = inH
    outW = inW
    #메모리 할당
    outImage = [[[0 for _ in range(outW)] for _ in range(outH)] for _ in range(3)]

    ## ** 찐 영상처리 알고리즘 ** ##
    avg = findavg(inImage)
    for rgb in range(3):
        for i in range(inH):
            for k in range(inW):
                if inImage[rgb][i][k] > avg :
                    outImage[rgb][i][k] = 255
                else:    
                    outImage[rgb][i][k] = 0

    ###########################
    displayImage()


    
#흑백3 : 중앙값
def BWImage3():
    global window, canvas, paper, filename
    global inImage, outImage, inH, inW, outH, outW
    #중요! 출력이미지의 크기를 결정 -> 알고리즘에 따라서
    outH = inH
    outW = inW
    #메모리 할당
    outImage = [[[0 for _ in range(outW)] for _ in range(outH)] for _ in range(3)]

    ## ** 찐 영상처리 알고리즘 ** ##
    median = findmedian(inImage)            
    for rgb in range(3):
        for i in range(inH):
            for k in range(inW):
                if inImage[rgb][i][k] > median :
                    outImage[rgb][i][k] = 255
                else:    
                    outImage[rgb][i][k] = 0

    ###########################
    displayImage()
    
#이동
def moveImage() :
    global window, canvas, paper, filename
    global inImage, outImage, inH, inW, outH, outW
    # 중요! 출력이미지의 크기를 결정 --> 알고리즘에 따라서...
    outH = inH
    outW = inW
    # 메모리 할당
    outImage = [[[0 for _ in range(outW)] for _ in range(outH)] for _ in range(3)]
    ## ** 찐 영상처리 알고리즘 ** ##
    xVal = askinteger("X값", "")
    yVal = askinteger("Y값", "")
    for rgb in range(3):
        for i in range(inH) :
            for k in range(inW) :
                if ( 0 <= i+xVal < outH ) and ( 0 <= k+yVal < outW) :
                    outImage[rgb][i+xVal][k+yVal] = inImage[rgb][i][k]
        ##############################
    displayImage()
    
#축소    
def zoomOutImage():
    global window, canvas, paper, filename
    global inImage, outImage, inH, inW, outH, outW
    scale = askinteger("축소비율", "") #축소비율은 미리 알아야함
    #중요! 출력이미지의 크기를 결정 -> 알고리즘에 따라서
    outH = inH//scale
    outW = inW//scale
    #메모리 할당
    outImage = [[[0 for _ in range(outW)] for _ in range(outH)] for _ in range(3)]

    ## ** 찐 영상처리 알고리즘 ** ##
    for rgb in range(3):
        for i in range(inH):
            for k in range(inW):
                outImage[rgb][i//scale][k//scale] = inImage[rgb][i][k]

    ###########################
    displayImage()

#확대    
def zoomInImage():
    global window, canvas, paper, filename
    global inImage, outImage, inH, inW, outH, outW
    scale = askinteger("확대비율", "") #축소비율은 미리 알아야함
    #중요! 출력이미지의 크기를 결정 -> 알고리즘에 따라서
    outH = inH*scale
    outW = inW*scale
    #메모리 할당
    outImage = [[[0 for _ in range(outW)] for _ in range(outH)] for _ in range(3)]

    ## ** 찐 영상처리 알고리즘 ** ##
      #보간 적용 x 확대 알고리즘
#     for i in range(inH):
#         for k in range(inW):
#             outImage[i*scale][k*scale] = inImage[i][k]
      #보간 적용 o 확대 알고리즘
    for rgb in range(3):
        for i in range(outH):
            for k in range(outW):
                outImage[rgb][i][k] = inImage[rgb][i//scale][k//scale]
    ###########################
    displayImage()

#회전
def rotateImage():
    global window, canvas, paper, filename
    global inImage, outImage, inH, inW, outH, outW
    angle = askinteger("각도", "0~360")
    #중요! 출력이미지의 크기를 결정 -> 알고리즘에 따라서
    outH = inH
    outW = inW
    #메모리 할당
    outImage = [[[0 for _ in range(outW)] for _ in range(outH)] for _ in range(3)]

    ## ** 찐 영상처리 알고리즘 ** ##
    radian = angle * math.pi / 180.0 #math.pi = 3.141592...
    
    cx = inH//2
    cy = inW//2
    
    for rgb in range(3):
        for i in range(outH):
            for k in range(outW):
                oldI = int(math.cos(radian)*(i-cx) + math.sin(radian)*(k-cy))+cx
                oldK = int(-math.sin(radian)*(i-cx) + math.cos(radian)*(k-cy))+cy

                if (0 <= oldI < inH) and (0 <= oldK < inW):
                    outImage[rgb][i][k] = inImage[rgb][oldI][oldK]

    ###########################
    displayImage()

#감마연산
def gammaImage() :
    global window, canvas, paper, filename
    global inImage, outImage, inH, inW, outH, outW
    
    #중요! 출력이미지의 크기를 결정
    outH = inH
    outW = inW
    #메모리 할당
    outImage = [[[0 for _ in range(outW)] for _ in range(outH)] for _ in range(3)]
    
    ## ** 찐 영상처리 알고리즘 ** ##
    inImage2 = copy.deepcopy(inImage)
    gamma = 1.5
    for rgb in range(3):
        for i in range(inH):
            for k in range(inW):
                outImage[rgb][i][k] = int(np.power(inImage2[rgb][i][k] / 255.0, gamma) * 255)
                
    displayImage()

#파라볼라연산
def paraImage():
    global window, canvas, paper, filename
    global inImage, outImage, inH, inW, outH, outW
    # 중요! 출력 이미지의 크기를 결정 --> 알고리즘에 따라서...
    outH = inH;  outW = inW;
    # 메모리 할당
    outImage = [[[0 for _ in range(outW)] for _ in range(outH)] for _ in range(3)]
     ## ** 찐 영상처리 알고리즘 ** ##
    for rgb in range(3) :
        for i in range(inH):
            for k in range(inW):
                outImage[rgb][i][k] = int(255 * (inImage[rgb][i][k]/128 - 1) **2)
    displayImage()
 




 ### 추가 함수 ###
#흑백2-1 : 평균값 찾기
def findavg(inImage):
    hap = 0
    for rgb in range(3): 
        for i in range(inH):
            for k in range(inW):
                hap += inImage[rgb][i][k]
    avg = hap / (inW * inH * 3)
    return avg

#흑백3-1 : 1차원 배열로 변환    
def imageto1array(inImage):
    array = []
    for rgb in range(3):
        for i in range(inH):
            for k in range(inW):
                array.append(inImage[rgb][i][k])
    return array

#흑백3-2 : 퀵정렬
def quick_sort2(arr):
    if len(arr) <= 1:
        return arr

    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    equal = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]

    return quick_sort2(left) + equal + quick_sort2(right)

#흑백3-3 : 중앙값 반환
def findmedian(inImage):
    image2 = imageto1array(inImage)
    #quick_sort(image2, 0, len(image2)-1)
    image3 = quick_sort2(image2)
    length = len(image3)
    if length % 2 == 1:
        median = image3[length // 2]
    else:
        mid1 = image3[length // 2]
        mid2 = image3[(length // 2) - 1]
        median = (mid1 + mid2) / 2
    return median


## 전역 변수부
window, canvas, paper = None, None, None 
filename = ""
inImage, outImage = None, None
inH, inW, outH, outW = 0, 0, 0, 0

## 메인 코드부
window = Tk()
window.geometry("300x300") 
window.title("GrayScale Image Processing (Beta 1)")

mainMenu = Menu(window) #메뉴의 틀
window.config(menu=mainMenu)

fileMenu = Menu(mainMenu) #상위 메뉴(파일)
mainMenu.add_cascade(label="파일",menu=fileMenu) #파일 누르면 펼쳐지는 거
fileMenu.add_command(label="열기", command=loadImage) #펼쳐지는 거 안의 명령
fileMenu.add_command(label="저장", command=saveImage)
fileMenu.add_separator() #줄 긋기
fileMenu.add_command(label="종료", command=None)

image1Menu = Menu(mainMenu) #상위 메뉴(파일)
mainMenu.add_cascade(label="화소점 처리", menu=image1Menu)
image1Menu.add_command(label="동일영상",command=equalImage)
image1Menu.add_command(label="그레이스케일",command=grayImage)
image1Menu.add_command(label="반전",command=reverseImage)
image1Menu.add_command(label="밝게/어둡게",command=addImage)
image1Menu.add_command(label="흑백1",command=BWImage1)
image1Menu.add_command(label="흑백2",command=BWImage2)
image1Menu.add_command(label="흑백3",command=BWImage3)
image1Menu.add_command(label="감마연산",command=gammaImage)
image1Menu.add_command(label="파라볼라연산",command=paraImage)

image2Menu = Menu(mainMenu) #상위 메뉴(파일)
mainMenu.add_cascade(label="기하학 처리",menu=image2Menu)
image2Menu.add_command(label="이동",command=moveImage)
image2Menu.add_command(label="축소",command=zoomOutImage)
image2Menu.add_command(label="확대",command=zoomInImage)
image2Menu.add_command(label="회전",command=rotateImage)

window.mainloop()



2023-06-28 21:59:07.501 python[76181:3379392] +[CATransaction synchronize] called within transaction
2023-06-28 21:59:07.556 python[76181:3379392] +[CATransaction synchronize] called within transaction
2023-06-28 21:59:07.611 python[76181:3379392] +[CATransaction synchronize] called within transaction
2023-06-28 21:59:07.639 python[76181:3379392] +[CATransaction synchronize] called within transaction
2023-06-28 21:59:07.658 python[76181:3379392] +[CATransaction synchronize] called within transaction
2023-06-28 21:59:07.672 python[76181:3379392] +[CATransaction synchronize] called within transaction
2023-06-28 21:59:07.692 python[76181:3379392] +[CATransaction synchronize] called within transaction
2023-06-28 21:59:07.706 python[76181:3379392] +[CATransaction synchronize] called within transaction
2023-06-28 21:59:07.725 python[76181:3379392] +[CATransaction synchronize] called within transaction
2023-06-28 21:59:07.742 python[76181:3379392] +[CATransaction synchronize] called within tr